In [2]:
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import sklearn.metrics as metrics
from sklearn import preprocessing

In [3]:
def libsvm_read(f):
    '''
    This just grabs Derek's svm_traintest.txt files
    and reads the data to that I can play with it easily. 
    This was more difficult than I expected, 
    and perhaps can be streamlined by writing better output files
    when extracting the parameters from the SWAMIS data.
    '''
    data = np.genfromtxt(f, dtype=str)
    # y are the labels, and I've set them to 1 for positive and -1 for negative instances
    # This is the convention in the SVM literature and lets me perform calculations on scikit-learn outputs
    y = data[:, 0].astype(int)
    y = (y - 5)/5

    NN = data[:, 1:].shape
    n = NN[0]
    m = NN[1]

    x = np.zeros(NN, dtype='<U22')
    for i in range(m):
        x[:, i] = np.char.replace(data[:, i+1], '{}:'.format(i+1), '')

    x = x.astype(np.float32)

    return x, y

In [4]:
def xi_alpha_est(C, x, y):
    '''
    The Xi-alpha estimators for the error rate, recall, precision, and F1 
    of the trained C-support vector classifier C are calculated

    input:
    C - C-svc object, trained on data x, y
    x - data array, contains all objects and their features
    y - label array, contains labels for all objects

    see Joachims, T.; 2000a
    https://www.cs.cornell.edu/people/tj/publications/joachims_00a.pdf
    '''
    dec = C.decision_function(x)
    eps = np.maximum(1 - y*dec, 0)

    '''
    Important point: alpha = 0 for all of the instances 
    that aren't support vectors (this isn't clear from the literature and documentation)
    Since scikit-learn doesn't store 0 alphas I had to correctly
    identify which instances are support vectors and then extract 
    their alpha values
    '''
    alpha = np.zeros_like(eps)
    ii_sv = np.where(x.all(axis=0) == C.support_vectors_.all(axis=0))
    alpha[ii_sv] = np.abs(C.dual_coef_[0])
    
    n = y.size
    inequa = 2*alpha + eps
    inequa.shape = n
    
    d = np.where((inequa >= 1))[0].size
    d_plus = np.where(np.logical_and(inequa >= 1, y == 1))[0].size
    d_min = np.where(np.logical_and(inequa <= -1, y == -1))[0].size
    n_plus = np.where((y == 1))[0].size

    Err = d/n
    Rec = 1 - d_plus/n_plus
    Prec = (n_plus - d_plus)/(n_plus - d_plus + d_min)
    F1 = 2*(n_plus - d_plus)/(2*n_plus - d_plus + d_min)

    return Err, Rec, Prec, F1


Pick your data set.
This *could* be made fancy, but I'm still working on implementing this in python.

In [5]:
X, Y = libsvm_read('svm_traintest_with_preB.txt')
#Y *= -1 #I'm testing the inverse problem to see how much the performance increases
mets = np.zeros((10))

Now I split the data once, to perform the coarse grid search and save those parameters. I will perform a finer search in the vicinity of these parameters to further optimize the problem. This is identical to the process going on inside the loop, except without the testing phase. The initial search parameters are the default that *libsvm* uses for their *easy.py* script.

In [24]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25, random_state = 123456789)

scale = preprocessing.StandardScaler().fit(x_train)
x_train = scale.transform(x_train)
x_test = scale.transform(x_test)

tuned_parameters = [{'kernel': ['rbf'], 'gamma': np.logspace(-15, 3, num = 25, base=2), 'C': np.logspace(-2, 15, num = 25, base=2)}]


CV = GridSearchCV(SVC(C=1), tuned_parameters, cv=10, scoring='accuracy')
CV.fit(x_train, y_train)
params = CV.best_params_

# 3 is an arbitrary choice, this can be changed 

loggmin = np.log2(params['gamma']) - 2
loggmax = loggmin + 4

logCmin = np.log2(params['C']) - 2
logCmax = logCmin + 4

print(params)

{'kernel': 'rbf', 'gamma': 0.026278012976678578, 'C': 1.0905077326652577}


In [30]:
for i in range(10):
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25, random_state=i)

    

    # Scaling the data to improve performance
    scale = preprocessing.StandardScaler().fit(x_train)
    x_train = scale.transform(x_train)
    x_test = scale.transform(x_test)


    # Parameters to search over
    # Could search over different kernels, but I'm not convinced
    # that will improve performance. It seems like gaussian kernels
    # are as good as we'll get for problems like this
    
    # The range here is chosen from a coarse grid search (over 10-15 decades) that I did over 
    # a single segmentation. My reasoning was that the random segmentations wouldn't change the 
    # behavior over so many decades, but would matter for a finer grid search
    # after finding the coarse best parameters. 
    tuned_parameters = [{'kernel': ['rbf'], 'gamma': np.logspace(-8, -5, num = 10, base=2), 'C': np.logspace(0, 4, num = 10, base=2)}]

    # Grid search over the parameters
    # cv is another free parameter that I tuned manually
    # This is another bottleneck: sometimes you can run into an issue
    # where there are no positive instances in a cross-validation
    # segmentation. More data will definitely improve this, but may also change
    # the best-fit for this value.
    CV = GridSearchCV(SVC(C=1), tuned_parameters, cv=10, scoring='accuracy')
    CV.fit(x_train, y_train)

    params = CV.best_params_
    #print(params)

    C = SVC(C = params['C'], gamma = params['gamma'])
    C.fit(x_train, y_train)

    #err, rec, prec, f1 = xi_alpha_est(C, x_train, y_train)

    preds = C.predict(x_test)

    

    mets[i] = metrics.accuracy_score(y_test, preds)
    #print(metrics.recall_score(y_test, preds), rec)
    #print(metrics.precision_score(y_test, preds), prec)
    #print(metrics.f1_score(y_test, preds), f1)
    #print(y_train.size, y_test.size)
    
print(mets.mean(), mets.std())
print(mets)


0.888888888889 0.0691661088777
[ 0.94444444  0.88888889  0.97222222  0.97222222  0.88888889  0.86111111
  0.94444444  0.80555556  0.75        0.86111111]
